# Pytorch Tensors - simple matrix operations and attributes

### Apart from the transpose, more matrix operations and attributes are presented here.

Some of the most commonly used operations in linear algebra, these are essential to "characterize" your matrix and its properties. 
- torch.inverse()
- torch.det()
- torch.matrix_rank()
- torch.diag()
- torch.reshape()

In [2]:
# Import torch and other required modules
import torch

## Function 1 - __[torch.inverse(input, out=None)](https://pytorch.org/docs/stable/torch.html#torch.inverse)__

A square matrix A of order <i>n</i> is said to be <i>invertible</i> if there is another square matrix B, in which:

$$AB = BA = I_{n}$$

In [21]:
# Example 1 - working
A = torch.tensor([[1., 2.], [2., 3.]])
print(A)

B = torch.inverse(A)
print(B)

print(torch.mm(A, B))

# We could also compare our multiplication with an identity matrix of order n:
# The @ operator denotes matrix multiplication and torch.eye(2) is a 2x2 Identity matrix
(A @ torch.inverse(A)) == torch.eye(2)

tensor([[1., 2.],
        [2., 3.]])
tensor([[-3.,  2.],
        [ 2., -1.]])
tensor([[1., 0.],
        [0., 1.]])


tensor([[True, True],
        [True, True]])

With the square matrix A, we calculated its inverse (matrix B), and by multiplying them we got a resulting matrix that is, as we wanted to prove, an identity matrix.

In [22]:
# Example 2 - working
C = torch.tensor([[3., 2.],
                  [4., 3.]])

D = torch.tensor([[3., 2.],
                  [4., 3.]])

M = C @ D
Mi = torch.inverse(M)
print(Mi)

Ci = torch.inverse(C)
Di = torch.inverse(D)
print(Ci @ Di)

# Since both tensors C and D are the same, we can very well expect that their product
# will be invertible as well. However, if we had used a comparison operator,
# the output would be False, which is due to the decimal precision difference in both operations
torch.inverse(M) == torch.inverse(C) @ torch.inverse(D)

tensor([[ 17.0001, -12.0001],
        [-24.0002,  17.0001]])
tensor([[ 17., -12.],
        [-24.,  17.]])


tensor([[False, False],
        [False, False]])

As shown above, if we're given two matrices C and D (both of order n), then torch.mm(C, D) is invertible and:

$$(CD)^{⁻1} = C^{-1}D^{-1}$$

In [23]:
# Example 3 - breaking
E = torch.tensor([[1., 0.], [1., 0.]])
print(E)

F = torch.inverse(E)
print(F)

torch.mm(E, F)

tensor([[1., 0.],
        [1., 0.]])


RuntimeError: inverse_cpu: U(2,2) is zero, singular U.

Matrix E is not invertible, so it's not possible to calculate its inverse and generates a RuntimeError: inverse_cpu: U(2,2) is zero, singular U.

A system of linear equations, such as $$Ax = b$$ will be possible and determinate (it has a single, unique solution), if its coefficient matrix A is invertible, and its determinant is non-null. Being the case, the solution for x will be: $$x = A^{-1}b$$

## Function 2 - __[torch.det()](https://pytorch.org/docs/stable/torch.html#torch.det)__

A matrix is only invertible, if and only if its determinant is non-null. 
If you consider a square matrix *A = [[a, b], [c, d]]* and assume *a* is non-null, and apply the Gaussian algorithm, you will reach the conclusion that the determinant can be calculated as *ad-bc != 0*.

In [24]:
# Example 1 - working
A = torch.tensor([[1., 2.], [2., 3.]])
torch.det(A)

tensor(-1.)

Using the previous matrix A, that we know is invertible, its determinant will be:
$$1 \times 3 - 2 \times 2 = -1$$

In [25]:
# Example 2 - working
G = torch.tensor([[3., 2., 6.],
                  [5., 3., 2.],
                  [1., 4., 2.]])
torch.det(G)

tensor(80.)

For *n=3* tensors, the determinant can be calculated by somewhat "extending" the matrix on its right, by its first *n-1* columns, and adding same direction diagonals:

$$|A| = a_{11} a_{22} a_{33} + a_{12} a_{23} a_{31} + a_{13}  a_{21} a_{32}
−a_{11} a_{23} a_{32} − a_{12} a_{21} a_{33} − a_{13} a_{22} a_{31}$$


$$3 \times 3 \times 2 + 2 \times 2 \times 1 + 6 \times 5 \times 4 -
3 \times 2 \times 4 - 2 \times 5 \times 2 + 6 \times 3 \times 1 = 80$$

In [26]:
# Example 3 - breaking
H = torch.randn(3,2)
print(H)
torch.det(H)

tensor([[-0.1532, -0.1306],
        [ 0.5086, -0.4279],
        [-1.9572,  0.1699]])


RuntimeError: A must be batches of square matrices, but they are 2 by 3 matrices

We know from the documentation, that a square matrix is necessary to calculate *det()*. If the input tensor is not square then Pytorch will output: "RuntimeError: A must be batches of square matrices".


More on determinants:
- If $A_{n \times n}$ has one null row or null column then $|A|=0$
- $|A| = |A^{T}|$
- If $A$ is triangular (inferior or superior) then $|A| = \prod_{i=1,..,n}(A)_{ii}$

The determinant tells us things about the matrix that are useful in systems of linear equations, helps us find the inverse of a matrix, is useful in calculus and more.

## Function 3 - __[torch.matrix_rank(input, tol=None, symmetric=False)](https://pytorch.org/docs/stable/torch.html#torch.matrix_rank)__

The rank of a matrix A is the dimension of the vector space generated  by its columns and corresponds to the maximum number of linearly independent columns of A. It's usually represented by *car(A)*, *c(A)* or *rank(A)*. In PyTorch it is calculated by torch.matrix_rank(A).
Specifically, it's equal to the number of pivots obtained by Gaussian Elimination Algorithm to matrix A. It denotes the number of non-null lines of the corresponding triangular matrix U , and *rank(A) = rank(U)*.

In [27]:
# Example 1 - working
I = torch.tensor([[3., 0., 7.],
                  [5., 0., 2.],
                  [3., 0., 2.]])
torch.matrix_rank(I)
# torch.inverse(I)

tensor(2)

Since tensor I has a null column, we can check that *rank(I) < n* and that it is not invertible.
The same would occur if it had a null row.

In [28]:
# Example 2 - working
J = torch.tensor([[3., 3., 7.],
                  [5., 1., 4.],
                  [4., 8., 3.]])
torch.matrix_rank(J)

tensor(3)

Tensor J has *rank(J) = n*, so it is invertible and has *det(J) != 0*.

In [29]:
# Example 3 - breaking
K = torch.randn(3)
print(K)
torch.matrix_rank(K)

tensor([-1.5474,  1.6583, -0.3952])


RuntimeError: matrix_rank(Float{[3]}): expected a 2D tensor of floating types

The only break error I could find was when you input a non 2D tensor.

A square matrix A of n-order is said to be non-singular if *rank(A)=n*, and if *rank(A) < rank(A|b)* then the linear system of *Ax=b* is said to be an impossible system. Furthermore, a possible system is determined *rank(A) = rank(A|b)* if, and only if, there are no free variables, that is, if the number of pivots are exactly the same to the number of incognite variables and so *rank(A) = rank(A|b) = n*.

## Function 4 - __[torch.diag(input, diagonal=0, out=None)](https://pytorch.org/docs/stable/torch.html#torch.diag)__


If input is a vector (1-D tensor), then returns a 2-D square tensor with the elements of input as the diagonal.    If input is a matrix (2-D tensor), then returns a 1-D tensor with the diagonal elements of input.

The argument diagonal controls which diagonal to consider:
If diagonal = 0, it is the main diagonal.
If diagonal > 0, it is above the main diagonal.
If diagonal < 0, it is below the main diagonal.


In [30]:
# Example 1 - working
L = torch.randn(3,3)
print(L)
torch.diag(L)

tensor([[ 1.2320,  0.9977, -0.4161],
        [-1.6745, -1.4483, -0.9491],
        [-0.0571, -0.9602, -0.0216]])


tensor([ 1.2320, -1.4483, -0.0216])

The diagonal of matrix L is given.

In [31]:
# Example 2 - working
M = torch.randn(3)
print(M)
torch.diag(M, 1)

tensor([ 0.0860, -0.2763,  1.6309])


tensor([[ 0.0000,  0.0860,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.2763,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.6309],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

Explanation about example

In [32]:
# Example 3 - breaking (to illustrate when it breaks)
M = torch.tensor([
    [True, False, False],
    [False, False, False],
    [False, False, True]
], dtype=torch.bool)
print(M)
torch.diag(M)

tensor([[ True, False, False],
        [False, False, False],
        [False, False,  True]])


RuntimeError: _th_diag not supported on CPUType for Bool

If the input matrix is of boolean type, it will result in the above error.

## Function 5 - __[torch.reshape(input, shape)](https://pytorch.org/docs/stable/torch.html#torch.reshape)__

Returns the same elements of the input matrix, but in a different shape.

In [33]:
# Example 1 - working
N = torch.randn(8,2)
torch.reshape(N, (4,4))

tensor([[-0.0905, -0.4917,  0.0684,  1.4956],
        [ 0.3972, -0.3935, -0.1450, -0.6096],
        [ 0.8338,  1.2405,  1.6513,  0.0353],
        [ 1.4410, -0.8317,  1.3719, -2.0732]])

In [60]:
# Example 2 - working
O = torch.tensor([[0, 1], [2, 3]])
print(O)
print(O.dim())

P = torch.reshape(O, (-1,))
print(P)
print(P.dim())

tensor([[0, 1],
        [2, 3]])
2
tensor([0, 1, 2, 3])
1


You can use (-1,) as shape to "flatten" tensor O, from the documentation: "A single dimension may be -1, in which case it’s inferred from the remaining dimensions and the number of elements in input". It's interesting to check the matrix dimension change.

In [61]:
# Example 3 - breaking (to illustrate when it breaks)
Q = torch.randn(3,3)
torch.reshape(Q, (4,2))

RuntimeError: shape '[4, 2]' is invalid for input of size 9

You're not able to reshape a matrix if their number of elements are not the same.

Reshape will be a commonly used function, as well as the view() and squeeze() functions.

## Conclusion

This was mere attempt to address some simple tensor attributes/functions, which I believed to be common in linear algebra operations.

If any assertations are incorrect, I would very much appreciate your contact for correction.

This assignment is part of the DeepLearning: Zero to GANs course from Jovian.ml and FreeCodeCamp.org.

## Reference Links
Provide links to your references and other interesting articles about tensors
* Official documentation for `torch.Tensor`: https://pytorch.org/docs/stable/tensors.html
* https://towardsdatascience.com/pytorch-fundamentals-50af6121d4a3
* https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be
* https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
* https://towardsdatascience.com/pytorch-for-deep-learning-a-quick-guide-for-starters-5b60d2dbb564
* https://medium.com/quantyca/pytorch-tips-and-tricks-from-tensors-to-neural-networks-febc73b3e34c
* https://www.kdnuggets.com/2018/11/introduction-pytorch-deep-learning.html
* https://towardsdatascience.com/how-to-train-your-neural-net-tensors-and-autograd-941f2c4cc77c
* https://www.datacamp.com/community/tutorials/investigating-tensors-pytorch
* https://deeplizard.com/learn/video/fCVuiW9AFzY

In [69]:
!pip install jovian --upgrade --quiet

In [70]:
import jovian

In [71]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/dan-motp/01-tensor-operations-79c2d


'https://jovian.ml/dan-motp/01-tensor-operations-79c2d'